In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import OneHotEncoder
import sklearn.metrics as metrics
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn import *
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score
from pandas_profiling import ProfileReport

from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV


In [ ]:
#Installing via pip pandas profilinng
!pip install -U pandas-profiling

In [ ]:
#Read Train and Test Data 

data_train = pd.read_csv("data_train.csv")
data_test = pd.read_csv("data_test.csv")
data_train.head()

In [ ]:
# Pre-Processing data 
# One hot encode all categorical columns




In [ ]:
one_hot_cols=["if_marital","if_default","if_housing","if_loan","if_active_selling", "occupation_cleaner",
              "occupation_management","occupation_own-business","occupation_production","occupation_retired",
              "occupation_services","occupation_student","occupation_technician","occupation_unemployed",
              "education_primary","education_secondary","education_unknown"]


data_train[one_hot_cols] = data_train[one_hot_cols].apply(LabelEncoder().fit_transform)
print(data_train.head())

data_test[one_hot_cols] = data_test[one_hot_cols].apply(LabelEncoder().fit_transform)
print(data_test.head())

#data_train=pd.get_dummies(data_train, columns=one_hot_cols)
#data_test=pd.get_dummies(data_test, columns=one_hot_cols)






In [ ]:
# Use Pandas profiling to see feature stats for train
profile = ProfileReport(data_train, title='MPG Pandas Profiling Report', explorative = True)
profile

In [ ]:
# Use Pandas profiling to see feature stats for test
profile_test = ProfileReport(data_test, title='MPG Pandas Profiling Report', explorative = True)
profile_test

In [ ]:
X_train= data_train.loc[:, data_train.columns != 'label']
Y_train=data_train['label']
print(X_train.head())
print(Y_train)
X_test= data_test.loc[:, data_test.columns != 'label']
Y_test=data_test['label']

In [ ]:
# Pre-procesing train columns
scaler = preprocessing.MinMaxScaler(feature_range=(0, 1))
names = X_train.columns
d = scaler.fit_transform(X_train)
X_train = pd.DataFrame(d, columns=names)
print(X_train.head())


names = X_test.columns
d = scaler.fit_transform(X_test)
X_test = pd.DataFrame(d, columns=names)
print(X_test.head())

In [ ]:
print(Y_train.value_counts(normalize=True))
class_wt_zero= Y_train.value_counts(normalize=True)[0]
print(class_wt_zero)
print(Y_test.value_counts(normalize=True))
class_wt_one= Y_train.value_counts(normalize=True)[1]

clf = LogisticRegression(random_state=0, max_iter =50,class_weight={0.0:class_wt_zero,1.0:class_wt_one}).fit(X_train, Y_train)
Y_pred=clf.predict(X_test)
recall_score(Y_test, Y_pred)
print(classification_report(Y_test, Y_pred, labels=[0,1]))





In [ ]:

ran = RandomForestClassifier(max_depth=2, random_state=0, n_estimators=30,class_weight={0:class_wt_zero,1:class_wt_one} )
ran.fit(X_train, Y_train)
Y_pred=ran.predict(X_test)
recall_score(Y_test,Y_pred, zero_division=0)
importances = ran.feature_importances_
print(importances)
print(classification_report(Y_test, Y_pred, labels=[0,1]))
# summarize feature importance
for name, importance in zip(X_train, ran.feature_importances_):
    print(name, "=", importance)


In [ ]:
%matplotlib inline

# Figure Size
fig, ax = plt.subplots(figsize =(16, 9))
 
# Horizontal Bar Plot
ax.barh( X_train.columns, ran.feature_importances_)
 
# Remove axes splines
for s in ['top', 'bottom', 'left', 'right']:
    ax.spines[s].set_visible(False)
 
# Remove x, y Ticks
ax.xaxis.set_ticks_position('none')
ax.yaxis.set_ticks_position('none')
 
# Add padding between axes and labels
ax.xaxis.set_tick_params(pad = 5)
ax.yaxis.set_tick_params(pad = 10)
 
# Add x, y gridlines
ax.grid(b = True, color ='grey',
        linestyle ='-.', linewidth = 0.5,
        alpha = 0.2)
 
# Show top values
ax.invert_yaxis()
 
# Add annotation to bars
for i in ax.patches:
    plt.text(i.get_width()+0.2, i.get_y()+0.5,
             str(round((i.get_width()), 2)),
             fontsize = 10, fontweight ='bold',
             color ='grey')
 
# Add Plot Title
ax.set_title('Sports car and their price in crore',
             loc ='left', )
 
# Add Text watermark
fig.text(0.9, 0.15, 'Feature Importance', fontsize = 12,
         color ='grey', ha ='right', va ='bottom',
         alpha = 0.7)
 
# Show Plot
plt.show()

In [ ]:
#Grid Search CV on Tree algorithm

rfc=RandomForestClassifier(random_state=42)

param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv= 5)
CV_rfc.fit(X_train, Y_train)

print(CV_rfc.best_params_)

#{'criterion': 'gini', 'max_depth': 7, 'max_features': 'auto', 'n_estimators': 500}


In [ ]:

ran = RandomForestClassifier(max_depth=7, random_state=0, criterion='gini', max_features='auto', n_estimators=500)
ran.fit(X_train, Y_train)
Y_pred=ran.predict(X_test)
recall_score(Y_test,Y_pred, zero_division=0)
importances = ran.feature_importances_
print(importances)
print(classification_report(Y_test, Y_pred, labels=[0,1]))
# summarize feature importance
for name, importance in zip(X_train, ran.feature_importances_):
    print(name, "=", importance)


In [ ]:
# Random Search CV
from scipy.stats import randint as sp_randint

param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(ran, param_distributions=param_dist,
                                   n_iter=n_iter_search)

random_search.fit(X_train, Y_train)

print(random_search.best_params_)
